# Project Title
* Movie Market Analysis

# Project Overview

For this project, you will use exploratory data analysis to generate insights for a business stakeholder.

Business Problem
* Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

# Goals
  
The objective is to identify which types of movies perform best commercially and critically, and to translate these insights into clear business recommendations on what kinds of films the studio should produce.


## Data loading and initial inspection

Multiple datasets are used to capture box office performance, production budgets, genres, and audience and critic ratings.  
Before analysis, we inspect the structure and completeness of each dataset.


In [1]:
# Loading the libraries

import pandas as pd
import numpy as numpy
import matplotlib.pyplot as pyplot
import sqlite3

In [2]:
# loading the csv files
movie_gross = pd.read_csv('../zippedData/bom.movie_gross.csv.gz')
movies = pd.read_csv('../zippedData/tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')

# loading the tsv files
movie_info = pd.read_csv('../zippedData/rt.movie_info.tsv.gz', sep='\t')
reviews = pd.read_csv('../zippedData/rt.reviews.tsv.gz',sep='\t',encoding='latin1')


In [3]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [5]:
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [6]:
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [8]:
movie_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [9]:
# converting tsv files to csv  files 
movie_info.to_csv('rt.movie_info.csv',index=False)

reviews.to_csv('rt.reviews.csv',index=False)

In [10]:
# unzipping the database
import zipfile

with zipfile.ZipFile('../zippedData/im.db.zip','r') as zip_ref:
    zip_ref.extractall('zippedData')

In [11]:
# creating a database connection
conn = sqlite3.connect('im.db')
cursor = conn.cursor()

In [12]:
# importing the IMDB database

imdb_tables = """
    SELECT *
    FROM sqlite_master 
    WHERE type = 'table';
    """

pd.read_sql(imdb_tables,conn)

,type,name,tbl_name,rootpage,sql


## Methods

**MOVIE GROSS DATA**

checking and cleaning

In [13]:
# preview the box office dataset and check its dimensions
movie_gross.head(), movie_gross.shape

#The box office dataset contains 3,387 rows, with each row representing a single movie.  
#It includes five variables: the movie title (used as a join key), the distributing studio, domestic box office revenue, foreign box office revenue, and the year of release.


(                                         title studio  domestic_gross  \
 0                                  Toy Story 3     BV     415000000.0   
 1                   Alice in Wonderland (2010)     BV     334200000.0   
 2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
 3                                    Inception     WB     292600000.0   
 4                          Shrek Forever After   P/DW     238700000.0   
 
   foreign_gross  year  
 0     652000000  2010  
 1     691300000  2010  
 2     664300000  2010  
 3     535700000  2010  
 4     513900000  2010  ,
 (3387, 5))

In [14]:
# inspect data types and check for missing values in the box office dataset
movie_gross.info()

# The dataset shows some missing values and inconsistent data types.  
#While movie titles and release years are complete, studio information is missing for a small number of films.  
#Domestic gross revenue is mostly available, but foreign gross revenue is missing for a large share of movies and is currently stored as a string rather than a numeric value.  
#These issues need to be addressed before analysing financial performance.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


# clean revenue variables

We will:
* convert foreign_gross to numeric
* create a total gross revenue variable
* remove movies with no revenue information

After converting revenue variables to numeric format and removing movies with missing box office data, the dataset is reduced to 2,004 movies.  
These films have complete information on total box office revenue and form the basis for the financial performance analysis.


We intentionally dropped movies with no usable revenue

This is correct because:

success is defined by box office performance

keeping missing revenue would weaken conclusions

In [15]:
# convert foreign gross to numeric
movie_gross['foreign_gross'] = pd.to_numeric(movie_gross['foreign_gross'], errors='coerce')

# create total box office revenue
movie_gross['total_gross'] = movie_gross['domestic_gross'] + movie_gross['foreign_gross']

# drop movies without revenue data
movie_gross_clean = movie_gross.dropna(subset=['total_gross'])

movie_gross_clean.shape




(2004, 6)

In [16]:
# check for duplicate movie titles in the box office dataset
movie_gross_clean.duplicated(subset=['title']).sum()

# A duplicate check on movie titles in the box office dataset confirms that there are no duplicate entries.  
#Each movie appears only once, making the dataset suitable for merging with other movie-level data.



0

# Movie metadata- checking and cleaning

## Results

In [17]:
movies.head(), movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


(   Unnamed: 0            genre_ids     id original_language  \
 0           0      [12, 14, 10751]  12444                en   
 1           1  [14, 12, 16, 10751]  10191                en   
 2           2        [12, 28, 878]  10138                en   
 3           3      [16, 35, 10751]    862                en   
 4           4        [28, 878, 12]  27205                en   
 
                                  original_title  popularity release_date  \
 0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
 1                      How to Train Your Dragon      28.734   2010-03-26   
 2                                    Iron Man 2      28.515   2010-05-07   
 3                                     Toy Story      28.005   1995-11-22   
 4                                     Inception      27.920   2010-07-16   
 
                                           title  vote_average  vote_count  
 0  Harry Potter and the Deathly Hallows: Part 1           7.7       1078

In [18]:
# preview movie metadata and check dimensions
movies.head(), movies.shape

# The movie metadata dataset contains 26,517 movies and nine variables. with information on genres, language, popularity, release dates, and audience ratings.  
# For this analysis, the most relevant variables are the movie title, genre information, and audience ratings, which can be linked to box office performance to support production decisions.
# This dataset is much larger than the box office dataset
# We will not analyse all 26,000 movies
# We will later merge and filter to only movies with revenue data

(   Unnamed: 0            genre_ids     id original_language  \
 0           0      [12, 14, 10751]  12444                en   
 1           1  [14, 12, 16, 10751]  10191                en   
 2           2        [12, 28, 878]  10138                en   
 3           3      [16, 35, 10751]    862                en   
 4           4        [28, 878, 12]  27205                en   
 
                                  original_title  popularity release_date  \
 0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
 1                      How to Train Your Dragon      28.734   2010-03-26   
 2                                    Iron Man 2      28.515   2010-05-07   
 3                                     Toy Story      28.005   1995-11-22   
 4                                     Inception      27.920   2010-07-16   
 
                                           title  vote_average  vote_count  
 0  Harry Potter and the Deathly Hallows: Part 1           7.7       1078

In [19]:
# drop redundant index column
movies_clean = movies.drop(columns=['Unnamed: 0'])

movies_clean.head(), movies_clean.shape


(             genre_ids     id original_language  \
 0      [12, 14, 10751]  12444                en   
 1  [14, 12, 16, 10751]  10191                en   
 2        [12, 28, 878]  10138                en   
 3      [16, 35, 10751]    862                en   
 4        [28, 878, 12]  27205                en   
 
                                  original_title  popularity release_date  \
 0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
 1                      How to Train Your Dragon      28.734   2010-03-26   
 2                                    Iron Man 2      28.515   2010-05-07   
 3                                     Toy Story      28.005   1995-11-22   
 4                                     Inception      27.920   2010-07-16   
 
                                           title  vote_average  vote_count  
 0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
 1                      How to Train Your Dragon           7.7      

In [20]:
# check for missing values in the movie metadata dataset
movies_clean.isna().sum()

# A check for missing values confirms that the movie metadata dataset has no missing observations across all variables.  
# This indicates that the dataset is clean and reliable for use in subsequent analysis.

genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [21]:
# check for duplicate movie titles in the movie metadata dataset
movies_clean.duplicated(subset=['title']).sum()

#A duplicate check reveals that 1,829 movie titles appear more than once in the metadata dataset.  
#This indicates that some titles are associated with multiple records, which could lead to ambiguous matches when merging with box office data.  
#Duplicates must therefore be resolved before proceeding with data integration.


1829

In [22]:
# inspect duplicated titles
duplicate_titles = movies_clean[movies_clean.duplicated(subset=['title'], keep=False)]

duplicate_titles[['title', 'release_date', 'vote_average']].head(100)

#Duplicate movie titles in the metadata dataset largely reflect films released in different years or remakes with the same name.  
#To ensure accurate merging with box office data, metadata records will be matched to box office entries using both movie title and release year.



,title,release_date,vote_average
3,Toy Story,1995-11-22,7.9
10,Toy Story 2,1999-11-24,7.5
11,Alice in Wonderland,2010-03-05,6.6
26,Bad Boys,1995-04-07,6.6
36,Robin Hood,2010-05-14,6.3
...,...,...,...
1274,On Strike for Christmas,2011-09-06,4.9
1275,Actresses,2010-05-22,6.8
1292,Altiplano,2010-06-07,5.5
1302,Rage,2010-05-01,4.3


In [23]:
# extract release year from release_date
movies_clean['release_year'] = pd.to_datetime(
    movies_clean['release_date'],
    errors='coerce'
).dt.year

movies_clean[['title', 'release_date', 'release_year']].head()

#Release year was successfully extracted from the movie metadata dataset.  
#This variable will later be used to accurately match movie records with box office data and resolve duplicate titles during merging.

,title,release_date,release_year
0,Harry Potter and the Deathly Hallows: Part 1,2010-11-19,2010
1,How to Train Your Dragon,2010-03-26,2010
2,Iron Man 2,2010-05-07,2010
3,Toy Story,1995-11-22,1995
4,Inception,2010-07-16,2010


From the preview:

These are legitimate movies, not messy duplicates

Titles repeat because:

same names exist across years (e.g. Robin Hood)

sequels share similar naming (Toy Story, Toy Story 2)

This dataset does not include release year as a clean numeric column, only release_date

Because our box office dataset already contains year, we can use that to select the correct movie record.

When merging movie metadata with box office data:

keep the metadata record whose release year matches the box office year

this avoids arbitrary deletion and preserves correctness

# Movie budget - data cleaning

In [24]:
# inspect column names in the movie budgets dataset
movie_budgets.columns

#An inspection of the movie budgets dataset shows that it contains key identifiers and financial variables, including movie titles, release dates, production budgets, and box office revenues.  
#These variables make the dataset suitable for analysing the relationship between production costs and financial performance.



Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [25]:
# check shape and data types of the movie budgets dataset
movie_budgets.info()

#The movie budgets dataset contains 5,782 movies and has no missing values across its variables.  
#However, production budget and revenue fields are stored as text, indicating that these variables will need to be converted to numeric format before analysis.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [26]:
## cleaning production budget and revenue data
#"""
#The production budget and revenue variables are currently stored as text, likely due to currency symbols and commas.  
#These variables are converted to numeric format to allow comparison between production costs and financial returns.
#"""

# remove currency symbols and commas, then convert to numeric
# remove currency symbols and commas, then convert to numeric

budget_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

for col in budget_cols:
    movie_budgets[col] = (
        movie_budgets[col]
        .astype(str)  # ensures .str works even if already numeric
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
    )
    movie_budgets[col] = pd.to_numeric(movie_budgets[col], errors='coerce')

movie_budgets[budget_cols].dtypes

#"""#After removing currency symbols and converting values to numeric format, production budget and revenue variables are now stored as numeric types.  
#This allows direct comparison between production costs and box office returns in later analysis.
#"""


production_budget    int64
domestic_gross       int64
worldwide_gross      int64
dtype: object

In [27]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [28]:
movie_budgets.isna().sum()

#No missing values

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [29]:
# check for duplicate movie titles in the budget dataset
movie_budgets.duplicated(subset=['movie']).sum()

#A duplicate check reveals that 84 movie titles appear more than once in the budget dataset.  
#These duplicates likely reflect multiple records for the same film or alternative budget and revenue estimates.  
#Rather than removing records prematurely, duplicates will be addressed during the merging stage using release year and revenue consistency.



84

In [30]:
# inspect data types and check for missing values in the box office dataset
movie_gross.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2032 non-null   float64
 4   year            3387 non-null   int64  
 5   total_gross     2004 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB


# Rotten tomatoes metadata

In [31]:
# preview Rotten Tomatoes movie information

"""
The Rotten Tomatoes movie information dataset was explored to assess whether it adds value beyond existing box office and audience rating data.  
While the dataset contains limited financial information, it provides MPAA content ratings that are useful for understanding audience targeting.  
As a result, only selected variables from this dataset will be considered in later analysis.
"""
movie_info.head(), movie_info.shape

(   id                                           synopsis rating  \
 0   1  This gritty, fast-paced, and innovative police...      R   
 1   3  New York City, not-too-distant-future: Eric Pa...      R   
 2   5  Illeana Douglas delivers a superb performance ...      R   
 3   6  Michael Douglas runs afoul of a treacherous su...      R   
 4   7                                                NaN     NR   
 
                                  genre          director  \
 0  Action and Adventure|Classics|Drama  William Friedkin   
 1    Drama|Science Fiction and Fantasy  David Cronenberg   
 2    Drama|Musical and Performing Arts    Allison Anders   
 3           Drama|Mystery and Suspense    Barry Levinson   
 4                        Drama|Romance    Rodney Bennett   
 
                             writer  theater_date      dvd_date currency  \
 0                   Ernest Tidyman   Oct 9, 1971  Sep 25, 2001      NaN   
 1     David Cronenberg|Don DeLillo  Aug 17, 2012   Jan 1, 2013       

In [32]:
movie_info.columns


Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

In [33]:
# check number of missing values in each column
"""
A missing value check shows that the Rotten Tomatoes movie information dataset has complete movie IDs, but several descriptive and financial fields are incomplete.  
Synopsis (62), rating (3), and genre (8) are mostly available, while director (199) and writer (449) are missing for some movies.  
Financial and studio-related fields have substantial missingness, including currency (1,220), box office (1,220), and studio (1,066), suggesting these variables may not be reliable for financial analysis.

"""
movie_info.isna().sum()


id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [34]:
"""
Although the Rotten Tomatoes dataset contains additional descriptive variables such as studio and box office revenue, these fields contain substantial missing values and are therefore not reliable for analysis.  
Financial performance is instead drawn from dedicated box office and budget datasets with more complete coverage.  
The Rotten Tomatoes data is used selectively to capture MPAA content ratings, which provide insight into audience targeting.

"""
"""
The MPAA content rating variable has only three missing values out of 1,560 movies.  
Given the negligible level of missingness, these observations do not materially affect the analysis and are excluded only where MPAA ratings are required.

"""

# retain only decision-relevant and relatively complete columns
movie_info = movie_info[['id', 'rating', 'genre', 'runtime', 'theater_date']]

# check missing values again after subsetting
movie_info.isna().sum()


id                0
rating            3
genre             8
runtime          30
theater_date    359
dtype: int64

In [35]:
"""
A duplicate check using the Rotten Tomatoes movie identifier confirms that there are no duplicate movie records in the dataset.  
Each row represents a unique film, making the dataset suitable for selective use in the analysis.

"""

# check for duplicate Rotten Tomatoes movie IDs
movie_info.duplicated(subset=['id']).sum()


0

# Rotten Tomatoes dataset: reviews dataset

In [36]:
reviews.head(),reviews.shape

(   id                                             review rating   fresh  \
 0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
 1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
 2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
 3   3  Continuing along a line introduced in last yea...    NaN   fresh   
 4   3             ... a perverse twist on neorealism...     NaN   fresh   
 
            critic  top_critic         publisher               date  
 0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
 1  Annalee Newitz           0           io9.com       May 23, 2018  
 2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
 3   Daniel Kasman           0              MUBI  November 16, 2017  
 4             NaN           0      Cinema Scope   October 12, 2017  ,
 (54432, 8))

In [37]:
"""
The Rotten Tomatoes reviews dataset contains 54,432 rows and eight variables, with each row representing an individual critic review rather than a movie.  
The dataset includes review text, critic ratings, Fresh/Rotten classification, critic and publisher information, and review dates.  
While most identifier and classification fields are complete, some review text and critic rating fields contain missing values, reflecting incomplete review records.

"""

# preview the Rotten Tomatoes reviews dataset
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [38]:
"""
The Rotten Tomatoes reviews dataset contains substantial missing values in critic ratings and review text.  
Rather than imputing these values, the analysis focuses on variables with complete coverage, such as Fresh/Rotten classification and top critic indicators, which provide a consistent measure of critic sentiment.

"""

# check missing values in the reviews dataset
reviews.isna().sum()


id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [39]:
"""
Duplicate reviews by the same critic for the same movie were identified and addressed.  
Reviews with missing critic identifiers were retained, as critic names are not required for sentiment aggregation.  
The cleaned dataset is therefore suitable for aggregating Fresh/Rotten sentiment at the movie level.

"""
# check for duplicate reviews by the same critic for the same movie
reviews.duplicated(subset=['id', 'critic']).sum()


2220

In [40]:
"""
After removing duplicate reviews by the same critic for the same movie, no duplicate records remain in the reviews dataset.  
This confirms that the dataset is clean and suitable for aggregation at the movie level.

"""

# remove duplicate reviews by the same critic for the same movie
reviews_clean = reviews.drop_duplicates(subset=['id', 'critic'])

#reviews_clean.shape
reviews_clean.duplicated(subset=['id', 'critic']).sum()


0

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3